In [1]:
#Importamos las librerias
import pandas as pd
import numpy as np

In [2]:
#Cargamos los archivos que tenemos que usar
games = pd.read_csv("Csv_limpios/steam_games_limpio.csv")
items = pd.read_csv("Csv_limpios/steam_items_limpio.csv")
reviews = pd.read_csv("Csv_limpios/steam_reviews_limpio.csv")

In [3]:
games

genres                title  price      id   año
0          Action  Lost Summoner Kitty   4.99  761140  2018
1          Casual  Lost Summoner Kitty   4.99  761140  2018
2           Indie  Lost Summoner Kitty   4.99  761140  2018
3      Simulation  Lost Summoner Kitty   4.99  761140  2018
4        Strategy  Lost Summoner Kitty   4.99  761140  2018
...           ...                  ...    ...     ...   ...
72706       Indie        Russian Roads   1.99  610660  2018
72707      Racing        Russian Roads   1.99  610660  2018
72708  Simulation        Russian Roads   1.99  610660  2018
72709      Casual  EXIT 2 - Directions   4.99  658870  2017
72710       Indie  EXIT 2 - Directions   4.99  658870  2017

[72711 rows x 5 columns]

In [4]:
#Renombramos la columna id por item_id para poder usar el merge
games.rename(columns={"id" : "item_id"}, inplace=True)

In [5]:
#Unimos las tablas
items_games = pd.merge(items, games, on="item_id", how="inner")
#Agrupamos por el nombre, el genero y el año y sumamos las horas jugadas
items_group = items_games.groupby(["user_id", "genres", "año"])["playtime_forever"].sum().reset_index()

In [6]:
#Eliminamos las columnas que tengan 0 horas jugadas
items_group = items_group[items_group["playtime_forever"] != 0].reset_index(drop=True)
items_group

user_id    genres   año  playtime_forever
0           --000--    Action  2009            5329.0
1           --000--    Action  2010              22.0
2           --000--    Action  2011            6522.0
3           --000--    Action  2012          109346.0
4           --000--    Action  2013             363.0
...             ...       ...   ...               ...
2859573  zzzmidmiss    Sports  2010             194.0
2859574  zzzmidmiss    Sports  2014              16.0
2859575  zzzmidmiss  Strategy  2010              10.0
2859576  zzzmidmiss  Strategy  2011              69.0
2859577  zzzmidmiss  Strategy  2012             923.0

[2859578 rows x 4 columns]

In [7]:
#Pasamos el df a csv
items_group = items_group.sample(n=15000, random_state=123)
items_group.to_csv("Csv_api/funcion2.csv", index=False)

Esta funcion la hice de prueba para despues adaptarla a la api

In [8]:
# def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas 
#jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
def UserForGenre(genero:str):
    #Filtramos por el genero ingresado
    generosdf = items_group[items_group["genres"] == genero]
    #Agrupamos por el nombre y los generos y sumamos las horas jugadas
    horas_por_persona = generosdf.groupby(["user_id", "genres"])["playtime_forever"].sum().reset_index()
    #Pasamos el resultado a una lista
    maximo = horas_por_persona.loc[horas_por_persona["playtime_forever"].idxmax()].to_list()
    #Filtramos por el nombre de la persona con mas horas
    horas_por_año = generosdf[generosdf["user_id"] == maximo[0]]
    return f"Usuario con mas horas para el genero {genero} es {maximo[0]} Horas totales: {maximo[2]}, horas por año:", horas_por_año[["año", "playtime_forever"]].reset_index(drop=True)

UserForGenre("Casual")

('Usuario con mas horas para el genero Casual es 76561198037264594 Horas totales: 46817.0, horas por año:',
     año  playtime_forever
 0  2012           46817.0)

In [9]:
def UserForGenre(genero: str):
    # Filtrar el DataFrame por el género dado
    genresdf = items_group[items_group['genres'] == genero]

    if genresdf.empty:
        return f"No hay datos para el género {genero}."

    # Encontrar al usuario con más horas jugadas para ese género
    top_user = genresdf.loc[genresdf['playtime_forever'].idxmax()]['user_id']

    # Crear una lista de acumulación de horas jugadas por año
    hours_by_year = genresdf.groupby('año')['playtime_forever'].sum().reset_index()
    hours_by_year = hours_by_year.rename(columns={'año': 'year', 'playtime_forever': 'Horas'})

    # Obtener las horas totales sumando la columna 'playtime_forever' en el DataFrame original
    total_hours = genresdf['playtime_forever'].sum()

    # Convertir el DataFrame a una lista de diccionarios
    hours_list = hours_by_year.to_dict(orient='records')

    # Crear el diccionario de retorno
    result = {
        "Género": genero,
        "Usuario con más horas jugadas": top_user,
        "Total de horas jugadas": total_hours,
        "Acumulación de horas jugadas por año": hours_list
    }

    return result

# Ejemplo de uso:
genero_buscar = "Indie"
resultados = UserForGenre(genero_buscar)
print(resultados)

{'Género': 'Indie', 'Usuario con más horas jugadas': 'Ender_Cat', 'Total de horas jugadas': 7786581.0, 'Acumulación de horas jugadas por año': [{'year': 1988, 'Horas': 21.0}, {'year': 1996, 'Horas': 10.0}, {'year': 1997, 'Horas': 93.0}, {'year': 1998, 'Horas': 225.0}, {'year': 1999, 'Horas': 3765.0}, {'year': 2001, 'Horas': 419.0}, {'year': 2002, 'Horas': 371.0}, {'year': 2003, 'Horas': 20898.0}, {'year': 2004, 'Horas': 25.0}, {'year': 2005, 'Horas': 1715.0}, {'year': 2006, 'Horas': 2438502.0}, {'year': 2007, 'Horas': 2133.0}, {'year': 2008, 'Horas': 25030.0}, {'year': 2009, 'Horas': 16793.0}, {'year': 2010, 'Horas': 45834.0}, {'year': 2011, 'Horas': 1390009.0}, {'year': 2012, 'Horas': 633490.0}, {'year': 2013, 'Horas': 944514.0}, {'year': 2014, 'Horas': 431299.0}, {'year': 2015, 'Horas': 787326.0}, {'year': 2016, 'Horas': 458118.0}, {'year': 2017, 'Horas': 585991.0}]}
